In [1]:
# coding: utf-8
import re 
import pandas as pd
import requests
import random
import json
from urllib import parse
from urllib import request
import time
from datetime import datetime
from datetime import timedelta
import os


header={
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',
    'Connection':'keep-alive'
}

def get_id_by_name(name):
    url = 'https://so.gushiwen.org/search.aspx?'
    dict1 ={'value': name}
    url_data = parse.urlencode(dict1) #unlencode()将字典{k1:v1,k2:v2}转化为k1=v1&k2=v2
    html = requests.get(url=url+url_data, headers=header).content.decode('utf-8')
    s = r'so.gushiwen.org\/authorv_(.*?)\.aspx'
    id = re.findall(s, html)
    return id[0]

def get_pages_by_name(name):
    try:
        url = 'https://so.gushiwen.org/authors/authorvsw_'+get_id_by_name(name)+'A1.aspx'
        html = requests.get(url=url, headers=header).content.decode('utf-8')
        s = r'sumPage\" style=\"display:none;\">(.*?)<\/label>'
        pages = re.findall(s, html)
        return pages[0]
    except Exception as e:
        return 0

    
def get_poet_by_dynasty(Dynasty):
    url = 'https://so.gushiwen.org/authors/Default.aspx?p=1&'
    dict1 ={'c': Dynasty}
    url_data = parse.urlencode(dict1) #unlencode()将字典{k1:v1,k2:v2}转化为k1=v1&k2=v2
    html = requests.get(url=url+url_data, headers=header).content.decode('utf-8')
    
    s0 = r'<div class=\"son1\"><h1>.*?\/ (.*?)页'
    pages = re.findall(s0, html)
    poet=[]
    for i in range(0,int(pages[0])):
        url = 'https://so.gushiwen.org/authors/Default.aspx?p='+str(i+1)+'&'
        dict1 ={'c': Dynasty}
        url_data = parse.urlencode(dict1) #unlencode()将字典{k1:v1,k2:v2}转化为k1=v1&k2=v2
        html = requests.get(url=url+url_data, headers=header).content.decode('utf-8')

        s = r'<a target=\"_blank\" style=.*?<b>(.*?)<\/b>'
        new = re.findall(s, html)
        print(new)
        poet = poet+new
        
    return poet

def get_total_poem_by_name(name,dynasty):
        poetry = pd.DataFrame(columns=['author','dynasty','title','content'])
        pages = get_pages_by_name(name)
        path= r'C:\\Users\\HP\\新建文件夹\\pa_chong\\poem\\朝代\\'+dynasty+r'\\'+name+'.csv'
        if int(pages)>0:
#             不存在文件夹自动生成
            if not os.path.exists(dynasty):
                os.mkdir(dynasty)
            poetry.to_csv(path,index=False,encoding="utf_8_sig")
            print("正在爬取词条    "+name)
        else:
            print(name+'    未找到词条')
        for i in range(0,int(pages)):
            time.sleep(0.2)
            url='https://so.gushiwen.org/authors/authorvsw_'+get_id_by_name(name)+'A'+str(i+1)+'.aspx'

            html = requests.get(url=url,headers=header).content.decode('utf-8')
            s1= r'<textarea(.*?)<\/textarea>'
            content = re.findall(s1, html)
            print(str(i)+" "+url+"\t"+str(len(content)-1))
            
            for j in range(0,len(content)-1):
                try:
                    item0=content[j].split('>')[1]
                    item1=item0.split('——')
                    # 获取内容
                    poem_content=item1[0]
                    #获取题目
                    poem_title=item1[1].split('《')[1].split('》')[0]
                    # 获取作者
                    poem_author = item1[1].split('《')[0].split('·')[1]
                    # 获取作者朝代
                    poem_author_dynasty = item1[1].split('《')[0].split('·')[0]

                    poetry = poetry.append({'author':poem_author,'dynasty':poem_author_dynasty,
                                                'title':poem_title,'content':poem_content},
                                                ignore_index=True)
                except Exception as e:
                    print("1Erroe:")
                    print(e)
                    continue
            poetry.to_csv(path,index=False,mode='a', header=False,encoding="utf_8_sig")
            poetry.drop(poetry.index,inplace=True)#清空c


In [2]:
dynastys=["先秦","两汉","魏晋","南北朝","隋代","唐代","五代","宋代","金朝","元代","明代","清代",]
for d in dynastys:
    poets=get_poet_by_dynasty(d)
    for p in poets:
        get_total_poem_by_name(p,d)
        print()
    print("-----------------------------------------------------------------------------------------------------------------------")

['屈原', '左丘明', '韩非', '宋玉', '列御寇', '庄周', '李耳', '孙武', '荀子', '李斯']
['曾参', '孔丘', '姜子牙', '荀况', '黄石公', '商鞅', '吴起', '孙膑', '周公旦', '晏婴']
['孔伋', '吕不韦', '尉缭', '公羊高', '谷梁赤', '司马穰苴', '庄忌', '公孙龙', '卜商', '宁戚']
正在爬取词条    屈原
0 https://so.gushiwen.org/authors/authorvsw_9c69482f885fA1.aspx	10
1 https://so.gushiwen.org/authors/authorvsw_9c69482f885fA2.aspx	9

正在爬取词条    左丘明
0 https://so.gushiwen.org/authors/authorvsw_1585930d4e76A1.aspx	10
1 https://so.gushiwen.org/authors/authorvsw_1585930d4e76A2.aspx	10
2 https://so.gushiwen.org/authors/authorvsw_1585930d4e76A3.aspx	10
3 https://so.gushiwen.org/authors/authorvsw_1585930d4e76A4.aspx	6

韩非    未找到词条

正在爬取词条    宋玉
0 https://so.gushiwen.org/authors/authorvsw_c364b4db5ff7A1.aspx	7

正在爬取词条    列御寇
0 https://so.gushiwen.org/authors/authorvsw_de8c4d88daecA1.aspx	4

正在爬取词条    庄周
0 https://so.gushiwen.org/authors/authorvsw_6e442cfc26e0A1.aspx	3

李耳    未找到词条

孙武    未找到词条

荀子    未找到词条

正在爬取词条    李斯
0 https://so.gushiwen.org/authors/authorvsw_c3783dd37722A1.aspx	1

曾参  

KeyboardInterrupt: 